In [6]:
!pip install yfinance
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Analyzing Tesla Historical Revenue Data and Building a Dashboard

## Question 1: Use yfinance to Extract Stock Data

In [7]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
tesla = yf.Ticker("TSLA")

In [9]:
tesla_data = tesla.history(period="max")

In [10]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


##  Question 2: Use Webscraping to Extract Tesla Revenue Data

In [11]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

In [14]:
html_data = requests.get(url).text

In [18]:
soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[]

In [29]:
print(soup.prettify())

<html>
 <head>
 </head>
 <body>
  <div style="margin: 50px auto; width: 50%; border: 1px solid #dfdfdf; padding: 20px 50px 30px 50px; font-family:helvetica;">
   <h1>
    We do not allow automated access to our servers.
   </h1>
   <h2>
    <p>
     Automated access to our data is prohibited by our data provider.
    </p>
    <p>
     If you are a user attempting to access the site via a browser, please follow this process to regain access:
    </p>
    <ul>
     <li>
      Go to
      <a href="https://whatismyipaddress.com/" rel="noopener noreferrer" target="_blank">
       whatismyipaddress
      </a>
      and obtain your IPv4 address
     </li>
     <li>
      Email us your IPv4 address at
      <a class="__cf_email__" data-cfemail="4821262e270825292b3a273c3a2d262c3b66262d3c" href="/cdn-cgi/l/email-protection">
       [email protected]
      </a>
     </li>
     <li>
      We will add you to our whitelist within 24 hours
     </li>
    </ul>
   </h2>
  </div>
  <script data-cfasync

In [30]:
print(len(soup.find_all("tbody")))


0


In [31]:
tbody_elements = soup.find_all("tbody")
if len(tbody_elements) > 1:
    for row in tbody_elements[1].find_all("tr"):
        col = row.find_all("td")
        date = col[0].text
        revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
else:
    print("The second <tbody> does not exist in the HTML.")


The second <tbody> does not exist in the HTML.


In [32]:
for tbody in tbody_elements:
    print(tbody.prettify())


In [33]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

tbody_elements = soup.find_all("tbody")
if len(tbody_elements) > 1:
    for row in tbody_elements[1].find_all("tr"):
        col = row.find_all("td")
        if len(col) > 1:  # Ensure there are enough columns
            date = col[0].text
            revenue = col[1].text.replace("$", "").replace(",", "")
            tesla_revenue = tesla_revenue.append({"Date": date, "Revenue": revenue}, ignore_index=True)
else:
    print("The second <tbody> does not exist or is not accessible.")


The second <tbody> does not exist or is not accessible.


## Question 3: Use yfinance to Extract Stock Data

In [20]:
gamestop = yf.Ticker("GME")


In [21]:
gamestop_data = gamestop.history(period="max")


In [22]:
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4: Use Webscraping to Extract GME Revenue Data

In [23]:
url1 = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

In [24]:
html_data = requests.get(url1).text


In [25]:
soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[]

In [26]:
soup = BeautifulSoup(html_data, "html.parser")
soup.find_all('title')

[]

In [35]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])


In [36]:
gme_revenue.tail()

,Date,Revenue


## Question 5: Plot Tesla Stock Graph

In [39]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [44]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Tesla Stock Data", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [45]:
make_graph(tesla_data, tesla_revenue, 'Tesla Stock Data')

C:\Users\20368329\AppData\Local\Temp\ipykernel_25768\2976135261.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\20368329\AppData\Local\Temp\ipykernel_25768\2976135261.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



## Question 6: Plot GameStop Stock Graph

In [46]:
make_graph(gamestop_data, gme_revenue, 'GameStop')


C:\Users\20368329\AppData\Local\Temp\ipykernel_25768\2976135261.py:3: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\20368329\AppData\Local\Temp\ipykernel_25768\2976135261.py:4: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



In [43]:
print("Completed")

Completed
